In [50]:
import os
import requests
import pandas as pd
from package.api import roster, status, player

In [51]:
league = os.environ.get('LEAGUE')
username = os.environ.get('USERNAME')

In [52]:
data = roster(league)

In [53]:
players = data[0]['players']
players

['10229',
 '11581',
 '11604',
 '11619',
 '4666',
 '4983',
 '5859',
 '5937',
 '6770',
 '7543',
 '7611',
 '8130',
 '8131',
 '8136',
 '8150',
 '8151',
 '9486',
 '9493',
 '9508',
 '9758']

In [54]:
data = player()

In [55]:
for i in players:
    print(data[i])

{'dl_trading_id': None, 'number': 4, 'stats_id': None, 'search_full_name': 'rasheerice', 'birth_state': None, 'position': 'WR', 'fantasy_positions': ['WR'], 'injury_notes': None, 'team': 'KC', 'oddsjam_id': 'D5BE2AF21C8F', 'yahoo_id': None, 'search_first_name': 'rashee', 'sportradar_id': 'cbd73c09-da07-48b0-a0d6-f248dfa631b6', 'injury_body_part': None, 'pandascore_id': None, 'espn_id': None, 'height': '73', 'practice_participation': None, 'player_id': '10229', 'depth_chart_position': 'SWR', 'full_name': 'Rashee Rice', 'weight': '204', 'years_exp': 1, 'sport': 'nfl', 'depth_chart_order': 3, 'birth_city': None, 'fantasy_data_id': None, 'status': 'Active', 'swish_id': 1173538, 'search_rank': 59, 'age': 24, 'search_last_name': 'rice', 'metadata': {'rookie_year': '2023'}, 'first_name': 'Rashee', 'rotowire_id': 16963, 'injury_start_date': None, 'college': 'SMU', 'birth_country': None, 'hashtag': '#rasheerice-NFL-KC-4', 'practice_description': None, 'high_school': 'Richland (TX)', 'birth_date

In [56]:
df = pd.DataFrame(data)

In [57]:
df2 = df.T

In [58]:
df2 = df2.reset_index()

In [63]:
df2 = df2.rename(columns={'index':'id'})

In [64]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10559 entries, 0 to 10558
Data columns (total 49 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      10559 non-null  object
 1   dl_trading_id           0 non-null      object
 2   number                  10260 non-null  object
 3   stats_id                2980 non-null   object
 4   search_full_name        10527 non-null  object
 5   birth_state             0 non-null      object
 6   position                10319 non-null  object
 7   fantasy_positions       10258 non-null  object
 8   injury_notes            220 non-null    object
 9   team                    3418 non-null   object
 10  oddsjam_id              3417 non-null   object
 11  yahoo_id                6750 non-null   object
 12  search_first_name       10527 non-null  object
 13  sportradar_id           10457 non-null  object
 14  injury_body_part        702 non-null    object
 15  pa

In [65]:
list = ['id', 'first_name', 'last_name', 'position', 'team', 'age' ,'depth_chart_order', 'search_rank']

In [66]:
df2[list]

,id,first_name,last_name,position,team,age,depth_chart_order,search_rank
0,3992,Dave,Caldwell,None,None,None,None,9999999
1,6744,Greg,Ward,WR,PHI,28,None,9999999
2,1591,Alvin,Bailey,G,None,29,None,9999999
3,918,Niles,Paul,TE,None,32,None,9999999
4,4810,Kwayde,Miller,OT,None,28,None,9999999
...,...,...,...,...,...,...,...,...
10554,1875,C.J.,Mosley,LB,NYJ,31,1,294
10555,7926,Carl,Tucker,TE,None,24,None,426
10556,8842,Malkelm,Morrison,CB,None,None,None,9999999
10557,11255,Nick,Amoah,OL,None,None,None,9999999
